In [ ]:
pip install  numdifftools

In [ ]:
#import packages
import numpy as np
from sklearn import linear_model
import random
from math import log
import matplotlib.pyplot as plt
import numdifftools as nd
from scipy.optimize import fmin_cobyla


In [ ]:
#Define funcitons that will be useful for algorithms simulations 

def Rademacher_matrix(d,n):
    """
    This fucntion generates a Rademacher matrix
    """
    return np.random.choice([-1, 1], size=(d,n))



def Rademacher_matrix_concatenated(d,n):
    """
    This function generates a Rademacher matrix and add a line of ones
    """
    Z=Rademacher_matrix(d,n)
    Last_line_of_ones = np.ones((1, Z.shape[1]))
    return (Z,np.concatenate((Z, Last_line_of_ones), axis=0))



def Lasso_reg(Y_tilde,Z):
    """
    This function gives the solution to the Lasso regression in a multivariate model
    """
    lasso = linear_model.LassoCV(cv=5)
    lasso.fit(Z,Y_tilde )
    g = lasso.coef_
    u=lasso.intercept_
    return(g,u)



def random_vector_unit_spehre(d):
    random_vector = np.random.normal(size=d)
    return(random_vector/np.linalg.norm(random_vector))



def random_unit_vector_on_S_t(d,S_t):
     """
     This function simulates a unit random vector on the spehre S_t
     """
     u_t=np.zeros(d)
     indices=np.array(S_t)
     u_t[indices]=np.random.normal(size=len(indices))
     return(u_t/np.linalg.norm(u_t))


def psi(x,a=2):
    return ((1/2*(a-1))*(np.linalg.norm(x, ord=a)**2))


def Delta_psi(psi,x,y,a=2):
    gradient_psi= nd.Gradient(psi)(x,a)
    return(psi(y,a)-psi(x,a)-( gradient_psi.T@(y-x)))



def constraint (x,B):
    return (np.linalg.norm(x,ord=1) - B)


def miror_descent_objectif_function(x,nu,g_t_tilde,x_t,psi,a=2):
    return(nu*g_t_tilde.T*(x-x_t)+Delta_psi(psi=psi,x=x,y=x_t,a=a))


def miror_descent_argmin(objectif_function,eta,g_t_tilde,x_t,constraint,psi,B,a=2):
    d= x_t.shape[0]
    x0 = np.random.normal(size=(d,))
    result =  fmin_cobyla(objectif_function, x0, args=(eta,g_t_tilde,x_t,psi,a),  cons=constraint,consargs=(3,))
    return(result)





    



    











     

In [ ]:
def GradiantEstimate(x_t_vect:np.ndarray,d,n,delta,f,S):
    """
    This function corresponds to the pseudo algorithme 1 defined in the paper
    """
    Z=Rademacher_matrix(d,n)
    y_t_vect=np.zeros(n)
    for i in range(n):
        z_i=Z[:, i]
        y_t_vect[i] = f(x_t_vect+delta*z_i,S) #Construct the vector y_t:= f(x_t+delta*z_i)+noise. We suppose that f returns the true output plus noise
    y_tilde=y_t_vect/delta
    (g,u)=Lasso_reg(Y_tilde=y_tilde,Z=Z.T)
    return(g,u)


def debiased_Lasso(x_t_vect:np.ndarray,d,n,delta,f,S):
     """
    This function returns the debiased lasso
    """

     Z=Rademacher_matrix(d,n)
     y_t_vect=np.zeros(n)
     ones=np.ones(n)
     for i in range(n):
        z_i=Z[:, i]
        y_t_vect[i] = f(x_t_vect+delta*z_i,S) #Construct the vector y_t:= f(x_t+delta*z_i)+noise. We suppose that f returns the true output plus noise
     y_tilde=y_t_vect/delta
     (g,u)=Lasso_reg(Y_tilde=y_tilde,Z=Z.T)
     g_tilde=g+((1/n)*Z@(y_tilde-u*ones))
     return(g_tilde)

    
def BGD(delta,T_prime,f,d,y_t,S_t,S,nu=1):
        for t_prime in range (T_prime):
             u_t=random_unit_vector_on_S_t(d=d,S_t=S_t) # select the unit vector u_t uniformly at random
             x_t=y_t+delta*u_t #update x_t at each step where x_t=y_t+delta*u_t.
             c_t=(f(x_t+delta*u_t,S)-f(x_t+delta*u_t,S))/(2*delta)
             y_t=y_t-(nu*c_t*len(S_t)*u_t)
        return x_t




def Successive_selection_algo(T,eta,delta,f,s,B,d,n,S,f_star=0,x_star=0,nu=1):
    x_t_vect=np.zeros(d)
    S_hat_t=[]
    S_hat_t_minus_1=[1]
    #chi_tilde={x for x in chi if np.linalg.norm(x,ord=1)<=B}
    T_prime=int(T/2)
    simple_regret=[]
    cumulative_regret=[]
    t=0
    x_t_cost=[]
    while len(S_hat_t)<s and t<s and S_hat_t!=S_hat_t_minus_1  : # setting  conditions to stop the while loop 
        
        t=t+1
        g_hat_t,u_t=GradiantEstimate(x_t_vect=x_t_vect,d=d,delta=delta,f=f,S=S,n=T_prime) # Estimate the gradient g_t 
        right_set=[i for i in range(d) if abs(g_hat_t[i]) >= eta]
        S_hat_t_minus_1=S_hat_t.copy() # update S_hat_minus_1
        S_hat_t=S_hat_t_minus_1 + (right_set) # Thresholding
        S_hat_t=list(set(S_hat_t)) #To remove duplicates from S_hat_t
        if len(S_hat_t)==0:
             break
        else:
            x_t_minus_one=x_t_vect.copy() # Keep a copy of x_t to use it for the output 
            x_t_vect=BGD(T_prime=T_prime,f=f,d=d,y_t=x_t_vect,S_t=S_hat_t,delta=delta,S=S,nu=nu) # performe the finite difference algorithme that returns x_t
            simple_regret.append(f(x_t_vect,S)-f_star)
            cumulative_regret.append(((sum(cumulative_regret)+f(x_t_vect,S))/(len(cumulative_regret)+1))-f_star)
            x_t_cost.append(np.linalg.norm(x_star-x_t_vect))
        

    if len(S_hat_t)==s:
         return(x_t_vect,simple_regret,cumulative_regret,x_t_cost)
    elif len(S_hat_t)==0: 
         print('In this case the algorithme did not make any discovery, it did not work')
         return([],[],[],[])
    else:
         return(x_t_minus_one,simple_regret,cumulative_regret,x_t_cost)
    

def First_order_optimization(B,T,n,d,psi,eta,delta,S,f,objectif_function,constraint,f_star=0,x_star=0):
     x_t_vect=np.zeros(d)
     T_prime=int(T/2)
     simple_regret=[]
     cumulative_regret=[]
     cost=[]
     for t in range(T_prime):
          g_t_tide=debiased_Lasso(x_t_vect=x_t_vect,d=d,n=n,delta=delta,f=f,S=S)
          x_t_vect=miror_descent_argmin(objectif_function=objectif_function,eta=eta,g_t_tilde=g_t_tide,x_t=x_t_vect,psi=psi,B=B,a=2,constraint=constraint)
          simple_regret.append(f(x_t_vect,S)-f_star)
          cumulative_regret.append(((sum(cumulative_regret)+f(x_t_vect,S))/(len(cumulative_regret)+1))-f_star)
          cost.append(np.linalg.norm(x_star-x_t_vect))
     return(x_t_vect,simple_regret,cumulative_regret,cost)


          

    


In [ ]:
# Functions that will be used for testing 
def f_test(x_t,noise=0):#S le support 
    return(np.linalg.norm(x_t)**2+noise*np.random.normal(0,1,1))
def vect_f_test(x_t, delta, d, n):
    y_t_vecteur = np.zeros(n)
    for i in range(n):
        y_t_vecteur[i] = f_test(x_t, delta, d)
    y_t = f_test(x_t=x_t, delta=0, d=d, noise=0)
    return y_t_vecteur, y_t
def f_test_S(X_t,S,noise=1):
    X_S=np.array([X_t[i] if i in S else 0 for i in range(len(X_t))])
    return(np.linalg.norm(X_S)**2+np.sum(X_S))+noise*np.random.normal(0,1,1)



In [ ]:
# Define functions that will be used to estimate hyperparameters

def logarithmic_grid_delta(sigma,s,T,d,H,nb_pnt):
    delta=(((sigma**2)*s*d)/(T*(H**2)))**(1/4)
    begin=delta/nb_pnt
    stop=delta*nb_pnt
    return(np.logspace(np.log10(begin), np.log10(stop), num=nb_pnt))


def logarithmic_grid_eta(C,d,n,nb_pnt):
    eta=(log(d)/n)**(1/2)
    begin=eta/nb_pnt
    stop=eta*nb_pnt
    return(np.logspace(np.log10(begin), np.log10(stop), num=nb_pnt))


def logarithmic_grid_nu(n,nb_pnt):
    nu=1/n
    begin= nu/nb_pnt
    stop= nu/nb_pnt
    return(np.logspace(np.log10(begin), np.log10(stop), num=nb_pnt))





In [ ]:
#precising the parameters 
d=100
n=50
delta=2
eta=2

x_t=np.random.binomial(1, 1/2,size=(d,))
#x_star = random.choices([1/2], k=d)
s=20
S= random.sample(range(90), s)
T=30
B=2




In [ ]:
x_star=np.zeros(d)
indices=np.array(S)
x_star[indices]=0.5

In [ ]:
sigma=1
H=-s
nb_pnt=10
C=5

In [ ]:
logarithmic_grid_eta=logarithmic_grid_eta(C=C,d=d,n=n,nb_pnt=nb_pnt)

In [ ]:
logarithmic_grid_delta=logarithmic_grid_delta(sigma=sigma,s=s,T=T,d=d,H=H,nb_pnt=nb_pnt)

In [ ]:
logarithmic_grid_nu=logarithmic_grid_nu(n=n,nb_pnt=nb_pnt)

In [ ]:
f_star=f_test_S(x_star,S,noise=0)

In [ ]:
x_t,simple_regret,cumulative_regret,x_cost=Successive_selection_algo(T=T,eta=eta,delta=delta,f=f_test_S,s=s,B=B,d=d,n=n,f_star=f_star,S=S,x_star=x_star)

In [ ]:
plt.figure(figsize=(10, 5), dpi=100)
plt.plot(list(range(1,len(cumulative_regret)+1)),np.log(cumulative_regret))
plt.title("Représentation du regret cumulatif ")  # Titre du graphique
plt.ylabel('log(regret cumulatif)')  # Titre de l'axe y
plt.xlabel('nombre itérations')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5), dpi=100)
plt.plot(list(range(1,len(simple_regret)+1)),np.log(simple_regret))
plt.title("Représentation du regret simple ")  # Titre du graphique
plt.ylabel('log(regret simple)')  # Titre de l'axe y
plt.xlabel('nombre itérations')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5), dpi=100)
plt.plot(list(range(1,len(x_cost)+1)),np.log(x_cost))
plt.title("|| x_star - x_t|| ")  # Titre du graphique
plt.ylabel('|| x_star - x_t||')  # Titre de l'axe y
plt.xlabel('nombre itérations')
plt.show()

In [ ]:
for delta in logarithmic_grid_delta:
    for eta in logarithmic_grid_eta:
        for nu in logarithmic_grid_nu:
            x_t,simple_regret,cumulative_regret,x_cost=Successive_selection_algo(T=T,eta=eta,delta=delta,f=f_test_S,s=s,B=B,d=d,n=n,f_star=f_star,S=S,x_star=x_star,nu=nu)
            if len(simple_regret)!=0 and len(simple_regret)!=1 :
                plt.figure(figsize=(10, 5), dpi=100)
                plt.title(f"Résultats pour delta:{delta} et eta:{eta} et nu:{nu}")  # Titre du graphique
                plt.ylabel('cost')  # Titre de l'axe y
                plt.xlabel('nombre itérations')
                plt.plot(list(range(1,len(simple_regret)+1)),np.log(list(map(abs, simple_regret))),color='blue',label='regret simple')
                plt.plot(list(range(1,len(cumulative_regret)+1)),np.log(list(map(abs, cumulative_regret))),color='red',label='regret cumulatif')
                plt.legend(loc='upper right')
                plt.show()

In [ ]:
x_t,simple_regret,cumulative_regret,x_cost=First_order_optimization(B=B,T=T,n=n,d=d,psi=psi,eta=eta,delta=delta,S=S,f=f_test_S,objectif_function=miror_descent_objectif_function,f_star=f_star,x_star=x_star,constraint=constraint)